## The Cursed Play

#### Import Text Adventure

In [31]:
from text_adventure_games import (
    games, parsing, actions, things, blocks, viz
)

### Locations

In [32]:
old_theater = things.Location(
    "The Old Theater",
    "You are standing in an old theater. The play is about to start."
)
playwrights_home = things.Location(
    "Playwright's Home",
    "You are standing in the home of a famous playwright."
)
bookstore = things.Location(
    "Ancient Bookstore",
    "You are standing in an ancient bookstore. The books are dusty and old."
)
    
death = things.Location(
    "The play was cursed.",
    "You are dead. GAME OVER."
)
death.set_property("game_over", True)


# Map of Locations
old_theater.add_connection("north", playwrights_home)
playwrights_home.add_connection("east", bookstore)

### Game Map

In [33]:
tempgame = games.Game(old_theater, things.Character(name="Temp Player", description="", persona=""))

from text_adventure_games.viz import Visualizer
viz = Visualizer(tempgame)
graph = viz.visualize()
graph

## Items

### Gettable Items

In [34]:
# Cursed Script: At the Old Theater, perhaps hidden or guarded by a theatrical gimmick.
# Mysterious Letter: Found in a secret compartment or puzzle box at the Playwright's Home.
# Ancient Book on Curses: In a puzzle-locked section of the Bookstore, requiring clever thinking to access.

cursed_script = things.Item(
    "script",
    "a cursed script",
    "A CURSED SCRIPT THAT ONCE PLAYED EVERYONE WILL DIE.",
)
old_theater.add_item(cursed_script)

book_on_curses = things.Item(
    "book on curses",
    "a book that contains knowledge to solve any curses",
    "A BOOK THAT CAN SAVE LIVES FROM CURSES.",
)

letter = things.Item(
    "letter",
    "a mysterious letter",
    "A MYSTERIOUS LETTER THAT CONTAINS INFORMATION ABOUT A BOOK OF CURSE.",
)

### Scenary Items

In [35]:
# A bookshelf in bookstore
bookshelf = things.Item(
    "bookshelf",
    "a bookshelf",
    "THE BOOKSHELF CONTAINS MANY BOOKS.",
)
bookshelf.set_property("gettable", False)
bookshelf.add_command_hint("browse books")
bookstore.add_item(bookshelf)

# A stage in old theater
stage = things.Item(
    "stage",
    "a stage",
    "THE STAGE IS SET FOR A PLAY.",
)
stage.set_property("gettable", False)
stage.add_command_hint("set the stage for the play")
old_theater.add_item(stage)

## Characters

### Player

In [36]:
# Player
player = things.Character(
    name="The detective",
    description="You are a seasoned detective with a keen eye for details.",
    persona="I solve mysteries.",
)

### Non-player Character

In [37]:
# The playwright at his house
playwright = things.Character(
    name="playwright",
    description="A secretive playwright",
    persona="Talk to me. But it doesn't mean I will tell you."
)
playwright.set_property("trust", False)
playwright.set_property("character_type", "human")
playwrights_home.add_character(playwright)

# The playwright has a mysterious letter
playwright.add_to_inventory(letter)

## Actions

In [38]:
class Give_Script (actions.Action):
    ACTION_NAME = "give"
    ACTION_DESCRIPTION = "Give something to someone"
    ACTION_ALIASES = ["hand", "show"]

    def __init__(self, game, command: str):
        super().__init__(game)
        give_words = ["give", "hand", "show"]
        command_before_word = ""
        command_after_word = command
        for word in give_words:
            if word in command:
                parts = command.split(word, 1)
                command_before_word = parts[0]
                command_after_word = parts[1]
                break
        self.giver = self.parser.get_character(command_before_word)
        self.recipient = self.parser.get_character(command_after_word)
        self.script: things.Item = self.parser.match_item(
                "script", self.parser.get_items_in_scope(self.giver)
            )

    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The script must be in the giver's inventory
        * The character must be at the same location as the recipient
        """
        if not self.was_matched(self.script, "I don't see it."):
            return False
        if not self.giver.is_in_inventory(self.script):
            self.parser.fail("You don't have it.")
            return False
        if not self.giver.location.here(self.recipient):
            self.parser.fail("They are not here.")
            return False
        return True

    def apply_effects(self):
        """
        Drop removes an item from character's inventory and adds it to the
        current location. (Assumes that the preconditions are met.)
        Giving the playwright the script gains trust.
        """
        self.giver.remove_from_inventory(self.script)
        self.recipient.add_to_inventory(self.script)
        self.recipient.set_property("trust", True)
        description = "The detective gave the script to the playwright. The playwright is now more trusting."
        self.parser.ok(description)


In [39]:
class Talk (actions.Action):
    ACTION_NAME = "talk"
    ACTION_DESCRIPTION = "Talk to someone"
    ACTION_ALIASES = ["speak"]

    def __init__(self, game, command: str):
        super().__init__(game)
        talk_words = ["talk", "speak"]
        command_before_word = ""
        command_after_word = command
        for word in talk_words:
            if word in command:
                parts = command.split(word, 1)
                command_before_word = parts[0]
                command_after_word = parts[1]
                break
        self.talker = self.parser.get_character(command_before_word)
        self.recipient = self.parser.get_character(command_after_word)
        
    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The recipient must trust the talker
        * The character must be at the same location as the recipient
        """
        if not self.recipient.get_property("trust"):
            self.parser.fail("The playwright doesn't trust you enough to talk to you.")
            return False
        if not self.talker.location.here(self.recipient):
            return False
        return True

    def apply_effects(self):
        """
        Removes inventory from the recipient and add it to the location.
        """
        letter = self.recipient.inventory['letter']
        self.recipient.remove_from_inventory(letter)
        self.recipient.location.add_item(letter)
        description = "The playwright takes out a letter and shows it to the detective."
        self.parser.ok(description)


In [40]:
class Browse_Books (actions.Action):
    ACTION_NAME = "browse books"
    ACTION_DESCRIPTION = "Browse books on the bookshelf."
    ACTION_ALIASES = []

    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.bookshelf = self.parser.match_item(
            "bookshelf", self.parser.get_items_in_scope(self.character)
        )
        self.letter = self.parser.match_item(
            "letter", self.parser.get_items_in_scope(self.character)
        )
        
    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be at the same location as the bookshelf
        * The character must have the mysterious letter in their inventory to find a useful book
        """
        if not self.bookshelf.location.here(self.character):
            return False
        if not self.letter:
            description = "You could not find any useful book."
            self.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        The character finds the Book on Curses.
        """
        self.bookshelf.location.add_item(book_on_curses)
        description = "You found a book on curses on the shelf."
        self.parser.ok(description)

In [41]:
class Read_Book (actions.Action):
    ACTION_NAME = "read book on curses"
    ACTION_DESCRIPTION = "read book on curses to learn the knowledge."
    ACTION_ALIASES = ["read book on curses", "learn"]

    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.book = self.parser.match_item(
            "book on curses", self.parser.get_items_in_scope(self.character)
        )
        
    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The book must exist
        * The character must be at the same location as the book
        """
        if not self.book:
            description = "You could not find any book."
            self.parser.fail(description)
            return False
        return True

    def apply_effects(self):
        """
        The character learns the counter curse
        """
        self.character.set_property("learned_counter_curse", True)
        description = "{name} learned the counter curse.".format(name=self.character.name)
        self.parser.ok(description)

In [42]:
class Set_Stage (actions.Action):
    ACTION_NAME = "Set the stage"
    ACTION_DESCRIPTION = "Set the stage for the play"
    ACTION_ALIASES = ["set stage"]

    def __init__(self, game, command):
        super().__init__(game)
        self.character = self.parser.get_character(command)
        self.stage = self.parser.match_item(
            "stage", self.parser.get_items_in_scope(self.character)
        )
        
    def check_preconditions(self) -> bool:
        """
        Preconditions:
        * The character must be at the same location as the stage
        """
        if not self.was_matched(self.stage, "There's no stage here."):
            return False
        if not self.stage.location.here(self.character):
            return False
        return True

    def apply_effects(self):
        """
        * If the character has learned the counter curse, the curse will be solved.
        * If the character has not learned the counter curse, the character dies.
        """
        if self.character.get_property("learned_counter_curse"):
            self.character.set_property("solved_curse", True)
            description = "The play is cursed. The detective learned the counter curse and people all survived."
            self.parser.ok(description)
        else:
            self.character.set_property("is_dead", True)
            description = "The play is cursed. The detective died."
            self.parser.ok(description)

## Play The Cursed Play

In [43]:
class CursedPlay(games.Game):
    def __init__(
        self, start_at: things.Location, player: things.Character, characters=None,
        custom_actions=None
    ):
        super().__init__(start_at, player, characters=characters, custom_actions=custom_actions)

    def is_won(self) -> bool:
        """ 
        Checks whether the game has been won. For Cursed Play, the game is won
        once the stage is set when any character has learned the counter curse.
        """
        for name, character in self.characters.items():
            if character.get_property("solved_curse"):
                msg = "{name} has solved the curse successfully! The play is on. You won."
                self.parser.ok(msg.format(name=character.name.title()))
                return True
        return False

In [44]:
characters = [playwright]
custom_actions = [Browse_Books, Read_Book, Talk, Set_Stage, Give_Script]

# The Game
game = CursedPlay(old_theater, player, characters=characters, custom_actions=custom_actions)

In [45]:
game.game_loop()

You are standing in an old theater. The play is about to start.
Exits:
North to Playwright's Home

You see:
 * a cursed script
 * a stage
        set the stage for the play


The detective got the script.
You are standing in the home of a famous playwright.
Exits:
South to The Old Theater
East to Ancient Bookstore


Characters:
 * A secretive playwright

The detective gave the script to the playwright. The playwright is now more
trusting.
The playwright takes out a letter and shows it to the detective.
The detective got the letter.
You are standing in an ancient bookstore. The books are dusty and old.
Exits:
West to Playwright's Home

You see:
 * a bookshelf
        browse books


You found a book on curses on the shelf.
The detective got the book on curses.
The detective learned the counter curse.
You are standing in the home of a famous playwright.
Exits:
South to The Old Theater
East to Ancient Bookstore


Characters:
 * A secretive playwright

You are standing in an old theater. Th